# Analyze the TIME / ESS distribution 

Randomly choose 25 samples and evaluate the model's Time / ESS 

### Import the libraries 

In [11]:
# Standard library imports
import sys

import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['KERAS_BACKEND'] = 'tensorflow'

# Third-party library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import arviz as az
import timeit

import scipy.stats as stats
from keras.models import Model as Model_nn
from keras.models import Sequential, load_model
from keras.layers import Dense, Concatenate
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Input, Dense, Add

#Try with TinyDA
import tinyDA as tda
from scipy.stats import multivariate_normal
from scipy.stats import uniform
from itertools import product


# Local module imports
sys.path.append('../../')
sys.path.append('../../solver')
#sys.path.append('./src/InverseProblems')
#sys.path.append('./src/utils')
from utils import * 
from plotting import *
from random_process import *
from model import *

### Choose the 25 random samples

In [12]:
n = 25 
np.random.seed(2109)
random_samples = np.random.randint(0, 160, n)
random_samples

array([57, 32, 55, 69,  3])

### Load the data and surrogate model 

In [13]:
# Extract test data for visualization or further processing
n_eig = 64
X_values = np.loadtxt('../../data/50-25-10/X_test_50resolution.csv', delimiter = ',')
y_values = np.loadtxt('../../data/50-25-10/y_test_50resolution.csv',delimiter = ',')

#### Load Low fidelity

In [14]:
# Choose the model parameters 
n_samples_lf = 16000
coeff_lf = 1e-08

In [15]:
# Initialize the neural network model
model_l = Sequential([
    Dense(256, input_shape=(X_values.shape[1],), activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(25, activation='linear')
])

model_l = load_model(f'../models/model_50resolution_{n_samples_lf}samples_1.keras')

/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### Load High Fidelity

In [16]:
# Choose the model parameters 
n_samples_lf_mf = 64000
coeff_lf_mf = 1e-09

# Initialize the neural network model
model_l_mf = Sequential([
    Dense(256, input_shape=(X_values.shape[1],), activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(25, activation='linear')
])

model_l_mf = load_model(f'../models/model_25resolution_{n_samples_lf_mf}samples_1.keras')

# Choose the model parameters 
n_samples = 16000
coeff = 1e-08

n_neurons = 256
# Initialize the neural network model
# Define the three branches of the model
input_params = Input(shape=(X_values.shape[1],))
input_pod = Input(shape=(y_values.shape[1],))

# Define the first branch (parameters)
x1 = Dense(n_neurons, activation='gelu')(input_params)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)

# Define the second branch (POD)
x2 = Dense(n_neurons, activation='gelu')(input_pod)

# # Define the second branch (POD)
# x3 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(w))(input_nn)

# Combine the outputs of the three branches
combined = Add()([x1,x2])
combined = Dense(n_neurons, activation='gelu')(combined)
output = Dense(25, activation='linear')(combined)

# Create the model
model_h = Model_nn(inputs=[input_params,input_pod], outputs=output)
model_h = load_model(f'..//models/model_2step_50-25resolution_{n_samples}samples_1.keras')

In [17]:
model_lf = lambda input : model_l(input.reshape(1,64)).numpy().reshape(25)
model_hf = lambda input: model_h([input.reshape(1,64), model_l_mf(input.reshape(1,64)).numpy()]).numpy().reshape(25)

### Time / ESS noise 0.001 and multiplicative coefficient 

In [21]:
noise = 0.001
scaling = 0.015 # 0.04
n_iter =  30000 #55000
burnin = 3000 #5000
thin = 20
sub_sampling = 1

Times = []
Time_ESS = []
ESS = []
i = 1

# Define the prior distribution and the proposal (common to all samples)
x_distribution = stats.multivariate_normal(mean = np.zeros(64), cov = np.eye(64))
my_proposal = tda.CrankNicolson(scaling=scaling, adaptive=False, gamma = 1.01, period=100)

for sample in random_samples:
    print('Sample = ', sample)
    x_true = X_values[sample]
    y_true = y_values[sample]

    y_observed = y_true + np.random.normal(scale=noise,size=y_true.shape[0])

    # LIKELYHOOD
    cov_likelihood = noise**2 * np.eye(25)
    y_distribution_coarse = tda.AdaptiveGaussianLogLike(y_observed, cov_likelihood*10)
    y_distribution_fine  = tda.GaussianLogLike(y_observed, cov_likelihood*10)

    # initialise the Posterior
    my_posterior_coarse = tda.Posterior(x_distribution, y_distribution_coarse, model_lf)
    my_posterior_fine = tda.Posterior(x_distribution, y_distribution_fine, model_hf)
    my_posteriors = [my_posterior_coarse, my_posterior_fine]

    # RUN THE MCMC
    start = timeit.default_timer()
    samples = tda.sample(my_posteriors, my_proposal, iterations=n_iter, n_chains=1, initial_parameters=np.zeros(64), subsampling_rate= sub_sampling, adaptive_error_model='state-independent')
    end = timeit.default_timer()

    # Remove the burnin and sub-sample
    idata = tda.to_inference_data(samples, level='fine')
    idata = idata.sel(draw=slice(burnin, None, thin), groups="posterior")
    ess = az.ess(idata)

    #Compute the time
    t = end-start
    Times.append(t)

    # Compute the mean ESS on the 64 parameters
    e = np.mean([ess.data_vars['x'+str(i)].values for i in range(64)])
    ESS.append(e)

    #Compute Time / ESS
    Time_ESS.append(t/e)
    
    print('Time:', t, '   ESS: ', e, '   Time/ESS: ',t/e )

# Save the results 
# Specify the folder path (assuming it already exists)
folder_path = './recorded_values'  # Replace with your actual path

# Save the file in the specified folder
file_path = os.path.join(folder_path, 'MDA_MF_2step_time_ess_001.npy')
np.save(file_path, Time_ESS)
file_path = os.path.join(folder_path, 'MDA_MF_2step_Times_001.npy')
np.save(file_path, Times)
file_path = os.path.join(folder_path, 'MDA_MF_2step_ESS_001.npy')
np.save(file_path, ESS)

Sample =  57
Sampling chain 1/1


Running chain, α_c = 0.319, α_f = 0.21:   2%|▏         | 19/1000 [00:00<00:11, 86.41it/s]/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/tinyDA/proposal.py:340: RuntimeWarning: overflow encountered in exp
  return np.exp(proposal_link.likelihood - previous_link.likelihood)
Running chain, α_c = 0.340, α_f = 0.30: 100%|██████████| 1000/1000 [00:11<00:00, 85.32it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 11.86524687500787    ESS:  3.5857771104810974    Time/ESS:  3.3089750169708485
Sample =  32
Sampling chain 1/1


Running chain, α_c = 0.320, α_f = 0.19: 100%|██████████| 1000/1000 [00:11<00:00, 87.45it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 11.45990124999662    ESS:  3.4507273236216887    Time/ESS:  3.32101037701494
Sample =  55
Sampling chain 1/1


...

Time: 12.05845379200764    ESS:  4.168628169231814    Time/ESS:  2.8926671563105004
Sample =  69
Sampling chain 1/1


Running chain, α_c = 0.270, α_f = 0.21: 100%|██████████| 1000/1000 [00:11<00:00, 86.31it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 11.607489916001214    ESS:  3.696851338118835    Time/ESS:  3.139831400931517
Sample =  3
Sampling chain 1/1


Running chain, α_c = 0.350, α_f = 0.29: 100%|██████████| 1000/1000 [00:12<00:00, 82.65it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 12.127233292005258    ESS:  3.875273631961117    Time/ESS:  3.12938761071903


### Time/ESS Higher Noise

In [20]:
noise = 0.01
scaling = 0.04
n_iter =  30000 #55000
burnin = 3000 #5000
thin = 20
sub_sampling = 1

Times = []
Time_ESS = []
ESS = []
i = 1

# Define the prior distribution and the proposal (common to all samples)
x_distribution = stats.multivariate_normal(mean = np.zeros(64), cov = np.eye(64))
my_proposal = tda.CrankNicolson(scaling=scaling, adaptive=False, gamma = 1.01, period=100)

for sample in random_samples:
    print('Sample = ', sample)
    x_true = X_values[sample]
    y_true = y_values[sample]

    y_observed = y_true + np.random.normal(scale=noise,size=y_true.shape[0])

    # LIKELYHOOD
    cov_likelihood = noise**2 * np.eye(25)
    y_distribution_coarse = tda.AdaptiveGaussianLogLike(y_observed, cov_likelihood)
    y_distribution_fine  = tda.GaussianLogLike(y_observed, cov_likelihood)
    # initialise the Posterior
    my_posterior_coarse = tda.Posterior(x_distribution, y_distribution_coarse, model_lf)
    my_posterior_fine = tda.Posterior(x_distribution, y_distribution_fine, model_hf)
    my_posteriors = [my_posterior_coarse, my_posterior_fine]

    # RUN THE MCMC
    start = timeit.default_timer()
    samples = tda.sample(my_posteriors, my_proposal, iterations=n_iter, n_chains=1, initial_parameters=np.zeros(64), subsampling_rate= sub_sampling, adaptive_error_model='state-independent')
    end = timeit.default_timer()

    # Remove the burnin and sub-sample
    idata = tda.to_inference_data(samples, level='fine')
    idata = idata.sel(draw=slice(burnin, None, thin), groups="posterior")
    ess = az.ess(idata)

    #Compute the time
    t = end-start
    Times.append(t)

    # Compute the mean ESS on the 64 parameters
    e = np.mean([ess.data_vars['x'+str(i)].values for i in range(64)])
    ESS.append(e)

    #Compute Time / ESS
    Time_ESS.append(t/e)
    
    print('Time:', t, '   ESS: ', e, '   Time/ESS: ',t/e , '     ', i,'/', len(random_samples))

    i = i+1

# Save the results 
# Specify the folder path (assuming it already exists)
folder_path = './recorded_values'  # Replace with your actual path

# Save the file in the specified folder
file_path = os.path.join(folder_path, 'MDA_MF_2step_time_ess_01.npy')
np.save(file_path, Time_ESS)
file_path = os.path.join(folder_path, 'MDA_MF_2step_Times_01.npy')
np.save(file_path, Times)
file_path = os.path.join(folder_path, 'MDA_MF_2step_ESS_01.npy')
np.save(file_path, ESS)

Sample =  57
Sampling chain 1/1


Running chain, α_c = 0.350, α_f = 0.29: 100%|██████████| 1000/1000 [00:12<00:00, 78.34it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 12.810139042005176    ESS:  4.091444278372272    Time/ESS:  3.1309577182122896       1 / 5
Sample =  32
Sampling chain 1/1


Running chain, α_c = 0.340, α_f = 0.33: 100%|██████████| 1000/1000 [00:13<00:00, 73.71it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 13.59484637499554    ESS:  3.8659825909307868    Time/ESS:  3.516530676285948       2 / 5
Sample =  55
Sampling chain 1/1


...

Time: 13.945761666982435    ESS:  4.786344051762926    Time/ESS:  2.9136563348064946       3 / 5
Sample =  69
Sampling chain 1/1


Running chain, α_c = 0.460, α_f = 0.35: 100%|██████████| 1000/1000 [00:17<00:00, 57.15it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 17.52990912500536    ESS:  4.050691863784756    Time/ESS:  4.327633331414729       4 / 5
Sample =  3
Sampling chain 1/1


Running chain, α_c = 0.390, α_f = 0.32: 100%|██████████| 1000/1000 [00:17<00:00, 56.41it/s]
/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/arviz/data/inference_data.py:157: UserWarning: qoi group is not defined in the InferenceData scheme
  warnings.warn(


Time: 17.766493667004397    ESS:  4.2453490689477285    Time/ESS:  4.184931174907622       5 / 5
